
https://stackoverflow.com/questions/43982864/allocating-production-jobs-for-films-for-multiple-film-manufacturing-machines-wi
I am trying to solve a problem which is somewhat similar to the bin packing problem. In this problem I have to assign jobs to film manufacturing machines where the machines have different film widths. All the machines must have the same amount of useage. Each roll has a specific width. Multiple jobs can be completed by the machine was long as the sum of all the job widths are less than the machine film width.

I have modeled it as a bin packing problem. Because I needed to make sure that all the machines have the same number of rolls I have the bins as a 2-d array where each type has multiple bins. In my formuation c is the capacity of the machine, x is a binary variable which specifies if a bin is being used, y is a binary variable which specifies if a roll is assigned to a bin. The overall objective is to minimize the waste generated. 

In [ ]:
from pulp import *
prob = LpProblem("Production Problem",LpMinimize)
capacity = [1350, 2100]
rolls = [970, 1050, 970, 1100, 1050, 500, 500, 500, 1050, 1350,1200, 370, 370]

I = range(2) # machine film width
J = range(10) # bins for each machine film width 
K = range(len(rolls)) # number of rolls in total 

# variable to determine wether a particular bin is used
x = LpVariable.dicts(name = "Bin", indexs = ((i,j) for i in I for j in J), lowBound = 0, upBound = 1, cat = 'Integer')
# variable to determine if roll is assigned to a particular bin 
y = LpVariable.dicts(name = "RollBin", indexs = ((i,j, k) for i in I for j in J for k in K), lowBound = 0, upBound = 1, cat = 'Integer')
w = LpVariable(name = 'wastage', lowBound=0)
for j in J:
    for i in I:
        firstPart = capacity[i] * x[(i,j)]
        secondPart = lpSum(rolls[k] * y[(i,j,k)] for k in K)
        w += firstPart - secondPart
prob+=w
#prob += lpSum(capacity[i] * x[(i,j)] - lpSum(rolls[k]*y[(i,j,k)] for k in K) for i in I for j in J)

for k in K:
    prob += lpSum([y[(i,j,k)] for i in I for j in J]) == 1

for k in K:
    prob += lpSum([rolls[k] * y[(i,j,k)] for i in I for j in J]) <= lpSum([capacity[i] * x[(i,j)] for i in I for j in J])

for i in I[:-1]:
    for j in J:
        prob += x[i,j] == x[i+1,j]

status = prob.solve()
print(prob)
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
print(value(prob.objective))
print(LpStatus[status])